In [ ]:
url = 'https://www.gpb.org/news/2020/08/03/fulton-countys-distribution-of-covid-19-relief-funds-unconscionable-local-mayor'

def download_article(url):
    """ download article from a url
    """
    from newspaper import Article
    article = Article(url)
    try:
        article.download()
        article.parse()
    except:
        print(f"{url} cannot be downloaded or parsed.")
        return None
    title = article.title
    text = article.text
    top_image = article.top_image
    movies = article.movies
    print(f"Downloaded an article with title: {title}..")
    result ={
        "title":title,
        "text":text,
        "top_image":top_image,
        "url":url,
#         "movies":movies,
    } 
    return result

In [ ]:
result = download_article(url)

In [ ]:
def get_urls_from_api(query,language,startdatetime,enddatetime,maxrecords=250):
    """get the urls based on the query keywords and time range
    """
    import urllib, json
    url = f"https://api.gdeltproject.org/api/v2/doc/doc?query={query}%20sourcelang:{language}&mode=artlist&STARTDATETIME={startdatetime}&ENDDATETIME={enddatetime}&MAXRECORDS={maxrecords}&format=json"
    data = urllib.request.urlopen(url).read().decode()
    data = json.loads(data)
    if not data:
        return []
    urls = []
    url_dic = {}
    for article in data['articles']:
        if article['url'] not in url_dic:
            urls.append(article['url'])
            url_dic[article['url']]=1
    return urls

In [ ]:
query = "BOTULISM"
language = "SWAHILI"
startdatetime = "20170101000000"
enddatetime = "20171231235959"
urls = get_urls_from_api(query,language,startdatetime,enddatetime)[:3]
urls

In [ ]:
def get_articles(query,language,startdatetime,enddatetime):
    urls = get_urls_from_api(query,language,startdatetime,enddatetime)
    articles = []
    for url in urls:
        article = download_article(url)
        if article:
            article["language"] = language
            articles.append(article)
    return articles

In [ ]:
import pandas as pd
languages = ['ENGLISH', 'chinese', 'SPANISH', 'ARABIC', 'FRENCH', 'PORTUGUESE', 'RUSSIAN','INDONESIAN', 'PERSIAN']
query = "BOTULISM"
startdatetime = "20170101000000"
enddatetime = "20171231235959"
# 'SWAHILI' is not supported
articles = []
for language in languages:
    articles += get_articles(query,language,startdatetime,enddatetime)
articles_df = pd.DataFrame(articles)
articles_df.to_csv(f"query_{query}_{startdatetime}_{enddatetime}.csv")